# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [96]:
import pandas as pd
import numpy as np
from scipy import stats
import scipy.stats as st
import math
import numpy as np


In [90]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [91]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [92]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


Question 1: 

I would use the concept of testing for differences between proportion with independent samples. 

3 required conditions for CLT: 
1/ Randomization Condition
2/ Independence Assumption
3/ 10% rule. The sample size should be no more than 10% of the population
4/ Large enough sample size

From the background of the survey data, researcher examined the racial discrimination level in USA by randomly assigning identical résumés to black-sounding or white-sounding names to the employers. So condition 1 is met. In addition, it's fair to assume that one call back does not affec the decision to call the next one. Therefore, condition 2 also met. 




In [93]:
print(data.shape)

(4870, 65)


In [ ]:
There are 4870 samples in this study. There are 156 million working people. So the sample is less than 10% of population. So Condition 
3 and 4 are also met. Therefore, CLT still apply in this investigation. 


Question 2: 

Null Hypothesis Ho: The race does not have any impact on the rate of callbacks for resume. In other word, black and white people have the same probablity either to have the call back or fail to have the call back. 

Alteranitve Hypothesis Ha: The race does have an impact on the rate of callbacks for resume. So one race will have more advantage than other for the rate of call back

Significant level is 5% 


    

In [97]:
w = data[data.race=='w']
b = data[data.race=='b']

# The total call back for black and white people
w_1=w[w.call==1]
b_1=b[b.call==1]


print ('number of black people in this study', len(b))
print ('number of call back for black people',len(b_1))
p_b=len(b_1)/len(b)

print ('number of white people in this study', len(w))
print ('number of call back for white people',len(w_1))
p_w=len(w_1)/len(w)



number of black people in this study 2435
number of call back for black people 157
number of white people in this study 2435
number of call back for white people 235


 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist 
    statistical approaches.

Frequentist approach: 95% Confident Interval
    

In [98]:
p=(len(b_1)+len(w_1))/(len(b)+len(w))

diff=(p_w-p_b)
moe=1.96*math.sqrt( (p_b*(1-p_b))/len(b) +  (p_w*(1-p_w))/len(w)              )

upper_lim=diff+moe
lower_lim=diff-moe


std_error=math.sqrt(p*(1-p)*(1/len(b)+1/len(w)))

z=((p_b-p_w)-0)/(std_error)

prob=(st.norm.sf(abs(z)))*2

print('The probablity difference on the rate call back between black and white people ',diff*100,'%')
print('Margin of error :',moe)
print('confident interval :', lower_lim, upper_lim)
print('P-value:', prob)


The probablity difference on the rate call back between black and white people  3.2032854209445585 %
Margin of error : 0.015255406349886438
confident interval : 0.016777447859559147 0.047288260559332024
P-value: 3.983886837585077e-05


In [ ]:
Because P-value < significant level, the null hypotheis can be rejected. Therefore, the race does have an impact on the rate of 
call back between black and white people 


In [48]:

def bootstrap_replicate_1d(data,n):
    
    bs_sample=np.random.choice(data,n)
    
    unique, counts = np.unique(bs_sample, return_counts=True)
    
    nocall_count=counts[0]
    call_count=counts[1]
    
    prob_nocall=nocall_count/n 
    prob_call=call_count/n
    
    return (prob_nocall,prob_call)

def draw_bs_reps(data,n,size=1):
    bs_replicates=np.empty(size)
    bs_prob_nocall=[]
    bs_prob_call=[]
    for i in range (size):
        prob_nocall,prob_call=bootstrap_replicate_1d(data,n)
        bs_prob_nocall.append(prob_nocall)
        bs_prob_call.append(prob_call)
        
    return (bs_prob_nocall,bs_prob_call)

In [101]:

# bootstrap sampling for black people 
bs_prob_b_nocall, bs_prob_b_call=draw_bs_reps(b['call'],len(b),10000)

# bootstrap sampling for white people 
bs_prob_w_nocall, bs_prob_w_call=draw_bs_reps(w['call'],len(w),10000)


bs_pb=np.mean(bs_prob_b_call)
bs_pw=np.mean(bs_prob_w_call)

p_overall=(bs_pb+bs_pw)/(len(data))

bs_diff=(bs_pw-bs_pb)
bs_moe=1.96*math.sqrt( (bs_pb*(1-bs_pb))/len(b) +  (bs_pw*(1-bs_pw))/len(w)              )

upper_lim=bs_diff+moe
lower_lim=bs_diff-moe

bs_std_error=math.sqrt(p_overall*(1-p_overall)*(1/len(b)+1/len(w)))

bs_z=((bs_pb-bs_pw)-0)/(bs_std_error)

bs_prob=(st.norm.sf(abs(bs_z)))*2

print('The probablity difference on the rate call back between black and white people ',(bs_diff)*100,'%')
print('Margin of error :',bs_moe)
print('confident interval :', lower_lim, upper_lim)
print('P-value:', bs_prob)

The probablity difference on the rate call back between black and white people  3.202160164271048 %
Margin of error : 0.015262759433865921
confident interval : 0.01676619529282404 0.04727700799259692
P-value: 0.0


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [ ]:
4. Write a story describing the statistical significance in the context or the original problem.
5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

In [ ]:
Question 4: Because P-value < significant level from both frequentist and boostrap approaches, the null hypotheis can be rejected. Therefore, the race does have an impact on the rate of 
call back between black and white people 

In [ ]:
Question 5: the analysis only suggest that race does have impact on the call back between black and white people. 
This study only investigate the relationship between race and callback. However, there are a lot of other parameters that could
affect the rate of callback like number of year experience, education, etc. I would say there seems a relationship between race 
and the rate of callback, but more in depth analysis will be required in order to arrive at the conclusion. 
